In [1]:
import tweepy as tw
import pandas as pd
from textblob import TextBlob

In [2]:
keys = pd.read_csv('keys.csv')

In [3]:
#Twitter API cred
consumerkey = keys['key'][0]
consumerSecret = keys['key'][1]
accessToken = keys['key'][3]
accessTokenSecret = keys['key'][4]

In [4]:
#Authentication Object
authenticate = tw.OAuth1UserHandler(consumerkey, consumerSecret)

#Set Access
authenticate.set_access_token(accessToken,accessTokenSecret)

#API Object
api = tw.API(authenticate, wait_on_rate_limit=True)

In [5]:
search_words = ["WakandaForever"]
limit = 10000

In [6]:
columns = ['User_name','tweets']
data = []

# Collect tweets
tweets = tw.Cursor(api.search_tweets,
              q=search_words, count=100, tweet_mode='extended', lang = 'en').items(limit)

for tweet in tweets:
    data.append([tweet.user.screen_name, tweet.full_text])

df = pd.DataFrame(data, columns=columns)

Rate limit reached. Sleeping for: 360


In [7]:
df.head()

,User_name,tweets
0,Drewboy14,RT @esquire: #BlackPanther made him an overnig...
1,Demarcus_Red,RT @getFANDOM: #WakandaForever blew everything...
2,iBE_ONLYME,RT @heroichollywood: We NEED Shuri &amp; Peter...
3,hiroshi00279644,RT @heroichollywood: Retweet if you’re hyped f...
4,YPinedodelarosa,RT @marvel_shots: ‘We Know What You Whisper’ \...


In [8]:
def count_letter(text):
    letter = len(text)
    return letter

In [9]:
df['letters_preprocess'] = df['tweets'].apply(count_letter)

In [10]:
df.head()

,User_name,tweets,letters_preprocess
0,Drewboy14,RT @esquire: #BlackPanther made him an overnig...,139
1,Demarcus_Red,RT @getFANDOM: #WakandaForever blew everything...,105
2,iBE_ONLYME,RT @heroichollywood: We NEED Shuri &amp; Peter...,100
3,hiroshi00279644,RT @heroichollywood: Retweet if you’re hyped f...,100
4,YPinedodelarosa,RT @marvel_shots: ‘We Know What You Whisper’ \...,85


In [11]:
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

def getAnalysis(score):
    if score > 0:
        return 1
    elif score == 0:
        return 4
    else:
        return 0

In [12]:
df['Polarity'] = df['tweets'].apply(getPolarity)

In [13]:
df.head()

,User_name,tweets,letters_preprocess,Polarity
0,Drewboy14,RT @esquire: #BlackPanther made him an overnig...,139,0.03125
1,Demarcus_Red,RT @getFANDOM: #WakandaForever blew everything...,105,0.00000
2,iBE_ONLYME,RT @heroichollywood: We NEED Shuri &amp; Peter...,100,0.00000
3,hiroshi00279644,RT @heroichollywood: Retweet if you’re hyped f...,100,0.00000
4,YPinedodelarosa,RT @marvel_shots: ‘We Know What You Whisper’ \...,85,0.00000


In [14]:
df['Label'] = df['Polarity'].apply(getAnalysis)
df['Label'].value_counts()

4    4739
1    3596
0    1665
Name: Label, dtype: int64

In [15]:
df.shape

(10000, 5)

In [16]:
df.head()

,User_name,tweets,letters_preprocess,Polarity,Label
0,Drewboy14,RT @esquire: #BlackPanther made him an overnig...,139,0.03125,1
1,Demarcus_Red,RT @getFANDOM: #WakandaForever blew everything...,105,0.00000,4
2,iBE_ONLYME,RT @heroichollywood: We NEED Shuri &amp; Peter...,100,0.00000,4
3,hiroshi00279644,RT @heroichollywood: Retweet if you’re hyped f...,100,0.00000,4
4,YPinedodelarosa,RT @marvel_shots: ‘We Know What You Whisper’ \...,85,0.00000,4


In [17]:
df.to_csv(r'tweets.csv')